In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["TERRA_CONFIG_PATH"] = "/home/sabri/code/spr-21/terra_config.json"
from robustnessgym.mosaic.contrib.wilds import WildsInputColumn, WildsDataPane

In [2]:
from robustnessgym.mosaic.contrib.wilds import WildsInputColumn, WildsDataPane
from robustnessgym.mosaic import DataPane
dp = WildsDataPane(
    "fmow", 
    root_dir="/home/common/datasets/", 
    use_transform=True,
    split="val"
)
#dp = DataPane.from_batch(dp[["y", "meta"]][0:10000])

In [3]:
import numpy as np
def func(x):
    return {
        "ones": np.ones(len(x)),
        "others": x + 1
    }
dp["y"].map(func, batched=True)

{'ones': NumpyArrayColumn(array([1., 1...., 1., 1., 1.])),
 'others': NumpyArrayColumn(array([ 5,  5..., 35, 35, 35]))}

In [5]:
import numpy as np
def func(x):
    return {
        "ones": np.ones(len(x["y"])),
        "others": x["y"] + 1
    }
dp.map(function=func, batched=True)

{'ones': NumpyArrayColumn(array([1., 1...., 1., 1., 1.])),
 'others': NumpyArrayColumn(array([ 5,  5..., 35, 35, 35]))}

In [7]:
import numpy as np
def func(x):
    return np.ones(len(x["y"]))
dp.map(function=func, batched=True)

NumpyArrayColumn(array([1., 1...., 1., 1., 1.]))

In [8]:
for x in dp[["y", "meta"]].batch():
    print('here')

In [18]:
import numpy as np
np.array_split(np.arange(100), 100 // 3)

[array([0, 1, 2, 3]),
 array([4, 5, 6]),
 array([7, 8, 9]),
 array([10, 11, 12]),
 array([13, 14, 15]),
 array([16, 17, 18]),
 array([19, 20, 21]),
 array([22, 23, 24]),
 array([25, 26, 27]),
 array([28, 29, 30]),
 array([31, 32, 33]),
 array([34, 35, 36]),
 array([37, 38, 39]),
 array([40, 41, 42]),
 array([43, 44, 45]),
 array([46, 47, 48]),
 array([49, 50, 51]),
 array([52, 53, 54]),
 array([55, 56, 57]),
 array([58, 59, 60]),
 array([61, 62, 63]),
 array([64, 65, 66]),
 array([67, 68, 69]),
 array([70, 71, 72]),
 array([73, 74, 75]),
 array([76, 77, 78]),
 array([79, 80, 81]),
 array([82, 83, 84]),
 array([85, 86, 87]),
 array([88, 89, 90]),
 array([91, 92, 93]),
 array([94, 95, 96]),
 array([97, 98, 99])]

In [3]:
from robustnessgym.mosaic.columns.abstract import AbstractColumn
dp["input"].batch.__func__ == AbstractColumn.batch

True

In [66]:
AbstractColumn.batch

<function robustnessgym.mosaic.columns.abstract.AbstractColumn.batch(self, batch_size: 'int' = 32, drop_last_batch: 'bool' = False, collate: 'bool' = True, *args, **kwargs)>

In [57]:
zip()

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [41]:
%%time
with dp.format(["y", "meta"]):
    out = dp.map(lambda x: x["y"] / 2, num_workers=0, batch_size=1000)


CPU times: user 65.2 ms, sys: 785 µs, total: 66 ms
Wall time: 63.4 ms


In [10]:
%%time
with dp.format(["y", "meta"]):
    out = dp.map(lambda x: x["y"] / 2)

here



CPU times: user 719 ms, sys: 1.25 s, total: 1.97 s
Wall time: 17.4 s


In [11]:
%%time
with dp.format(["y", "meta"]):
    out = dp.map(lambda x: x["y"] / 2)


CPU times: user 681 ms, sys: 1.35 s, total: 2.03 s
Wall time: 17.9 s


In [27]:
from spr.vision import Classifier
from spr.data.wilds import FMOW_CONFIG

classifier = Classifier(FMOW_CONFIG)

loading weights from /home/common/datasets/fmow_v1.1/models/fmow_erm_ID_seed0/best_model.pth...
/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [4]:
dp["input"]._materialize = False
small_dp = DataPane.from_batch(dp[:1000])
small_dp["input"]._materialize = True 
dp["input"]._materialize = True

In [28]:
import torch
from robustnessgym.mosaic.contrib.wilds import WildsInputColumn, WildsDataPane
from robustnessgym.mosaic import DataPane
from terra import Task

@Task.make_task
def score_fmow_with_mosaic(split: str="val", run_dir=None):

    classifier = Classifier(FMOW_CONFIG)
    dp = WildsDataPane(
        "fmow", 
        root_dir="/home/common/datasets/", 
        use_transform=True,
        split=split
    )

    classifier.eval()
    classifier.to(0)

    @torch.no_grad()
    def predict(batch: dict):
        out = torch.softmax(classifier(batch["input"].to(0)), axis=-1)
        return {"pred": out.cpu().numpy().argmax(axis=-1)}

    dp_w_preds = dp.update(
        function=predict,
        batch_size=128,
        batched=True,
        num_workers=8
    )
    return (dp_w_preds["pred"] == dp_w_preds["y"]).mean()[0]

In [29]:
score_fmow_with_mosaic(split="val")

[2021-04-30 00:15:34,641][WARNING][pip._internal.operations.freeze:72] :: Could not generate requirement for distribution -illow 8.2.0 (/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages): Parse error at "'-illow=='": Expected W:(abcd...)
task: score_fmow_with_mosaic, run_id=217
loading weights from /home/common/datasets/fmow_v1.1/models/fmow_erm_ID_seed0/best_model.pth...
/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)



Traceback (most recent call last):
  File "/home/sabri/code/terra/terra/__init__.py", line 230, in _run
    out = self.fn(**args_dict)
  File "<ipython-input-28-41cf8cb82d03>", line 25, in score_fmow_with_mosaic
    dp_w_preds = dp.update(
  File "/home/sabri/code/robustness-gym/robustnessgym/mosaic/datapane.py", line 908, in update
    output = new_dataset.map(
  File "/home/sabri/code/robustness-gym/robustnessgym/mosaic/datapane.py", line 978, in map
    else function(batch)
  File "/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 26, in decorate_context
    return func(*args, **kwargs)
  File "<ipython-input-28-41cf8cb82d03>", line 23, in predict
    return {"pred": out.cpu().numpy().argmax(axis=-1)}
KeyboardInterrupt



KeyboardInterrupt: 

In [29]:
len(dp_w_preds)

11327

In [49]:
score_fmow_with_mosaic.get_runs()

,id,module,fn,run_dir,status,notebook,start_time,end_time,hostname,python_version,platform,git_commit,git_dirty,slurm_job_id
0,215,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:49:15.647186,2021-04-29 21:50:28.611613,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
1,214,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,failure,False,2021-04-29 21:49:01.867664,2021-04-29 21:49:07.767328,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
2,213,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:46:07.806332,2021-04-29 21:46:53.003622,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
3,212,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:43:15.294843,2021-04-29 21:44:33.088492,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
4,211,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:37:07.563952,2021-04-29 21:38:31.318201,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
5,210,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,interrupted,False,2021-04-29 21:36:18.722919,2021-04-29 21:37:04.789272,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
6,209,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:29:55.705654,2021-04-29 21:31:06.848939,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
7,208,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,failure,False,2021-04-29 21:29:42.218594,2021-04-29 21:29:47.358688,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
8,207,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:28:20.296646,2021-04-29 21:29:03.381629,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
9,206,__main__,score_fmow_with_mosaic,/home/sabri/terra/slice/tasks/__main__/score_f...,success,False,2021-04-29 21:26:03.735195,2021-04-29 21:27:21.903094,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,006ff14f76ad080cc0f7b20387b225c2f85a4feb,True,None
